In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["PINECONE_API_KEY"]=os.getenv("PINECONE_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [4]:
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"
## initialize the pinecone client
pc=Pinecone(api_key=pinecone_api_key)

## create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct', ## sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud='aws', region="us-east-1"),
    )

In [5]:
index = pc.Index(index_name)
index

In [6]:
## vector embedding and sparse matrix
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder=BM25Encoder().default()
bm25_encoder

In [13]:
sentences=[
    "In 2023, I visited Purbacol",
    "In 2022, I visited Bashundhara",
    "In 2019, I visited Cox-s_xBazar",
]
## store the values to a json file
bm25_encoder.dump('bm25_values.json')

## load to your bm25encoder object
bm25_encoder = BM25Encoder().load('bm25_values.json')

In [14]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001D78E596BA0>, index=<pinecone.data.index.Index object at 0x000001D7EC7D9550>)

In [16]:
retriever.add_texts(
    [
    "In 2023, I visited Purbacol",
    "In 2022, I visited Bashundhara",
    "In 2019, I visited Cox-s_xBazar",
]
)

  0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 1/1 [00:02<00:00,  2.36s/it]


In [17]:
retriever.invoke("what city did i visited last?")

[Document(metadata={'score': 0.360141695}, page_content='In 2023, I visited Purbacol'),
 Document(metadata={'score': 0.33793509}, page_content='In 2019, I visited Cox-s_xBazar'),
 Document(metadata={'score': 0.311870873}, page_content='In 2022, I visited Bashundhara')]

In [18]:
retriever.invoke("what city did i visited first?")

[Document(metadata={'score': 0.359379232}, page_content='In 2023, I visited Purbacol'),
 Document(metadata={'score': 0.327061474}, page_content='In 2019, I visited Cox-s_xBazar'),
 Document(metadata={'score': 0.313121855}, page_content='In 2022, I visited Bashundhara')]